### Setup your Snowflake environment for chapter 3

In [1]:
import configparser
from snowflake.snowpark import Session
import time

In [2]:
# Connect to your snowflake account using your credentials
snowflake_credentials_file = 'snowflake_creds.config'

config = configparser.ConfigParser()
config.read(snowflake_credentials_file)

session = Session.builder.configs(dict(config['default'])).create()

# test the session
session.get_current_account()

'"PCB29551"'

In [3]:
# Make sure your Snowflake has the necessary resources initialized
snowflake_setup_file = "setup.sql"

with open(snowflake_setup_file, 'r') as file:
    sql_script = file.read()

sql_script_statements = sql_script.split(";")

print(session.sql(sql_script_statements[0]).collect())
print(session.sql(sql_script_statements[1]).collect())
print(session.sql(sql_script_statements[2]).collect())


[Row(status='PACKT already exists, statement succeeded.')]
[Row(status='CHAPTER_3 already exists, statement succeeded.')]
[Row(status='MY_STAGE already exists, statement succeeded.')]


# Start of Chapter 3 Code

In [4]:
session.file.put('./data/marketing_campaign.csv', 'PACKT.CHAPTER_3.MY_STAGE') 

[PutResult(source='marketing_campaign.csv', target='marketing_campaign.csv.gz', source_size=220189, target_size=0, source_compression='NONE', target_compression='GZIP', status='SKIPPED', message='')]

In [5]:
# list the files in the stage:
session.sql("list @PACKT.CHAPTER_3.MY_STAGE").collect()

[Row(name='my_stage/marketing_campaign.csv.gz', size=58880, md5='234f7588f443aeb2f965f86842263b5e', last_modified='Wed, 3 Jan 2024 03:49:04 GMT')]

In [6]:
import snowflake.snowpark.types as T 

purchase_history_schema = T.StructType(
    [
        T.StructField("ID", T.IntegerType()), 
        T.StructField("Year_Birth", T.IntegerType()),T.StructField("Education", T.StringType()), 
        T.StructField("Marital_Status", T.StringType()),T.StructField("Income", T.IntegerType()), 
        T.StructField("Kidhome", T.IntegerType()),T.StructField("Teenhome", T.IntegerType()), 
        T.StructField("Dt_Customer", T.DateType()),T.StructField("Recency", T.IntegerType()), 
        T.StructField("MntWines", T.IntegerType()),T.StructField("MntFruits", T.IntegerType()), 
        T.StructField("MntMeatProducts", T.IntegerType()),T.StructField("MntFishProducts", T.IntegerType()), 
        T.StructField("MntSweetProducts", T.IntegerType()),T.StructField("MntGoldProds", T.IntegerType()), 
        T.StructField("NumDealsPurchases", T.IntegerType()),T.StructField("NumWebPurchases", T.IntegerType()), 
        T.StructField("NumCatalogPurchases", T.IntegerType()),T.StructField("NumStorePurchases", T.IntegerType()), 
        T.StructField("NumWebVisitsMonth", T.IntegerType())
    ]
) 

In [7]:
session.sql("USE DATABASE PACKT").collect()
session.sql("USE SCHEMA PACKT.CHAPTER_3").collect
session.sql("TRUNCATE TABLE IF EXISTS PURCHASE_HISTORY").collect()

purchase_history = session.read\
.option("FIELD_DELIMITER", ',')\
.option("SKIP_HEADER", 1)\
.option("ON_ERROR", "CONTINUE")\
.schema(purchase_history_schema).csv("@PACKT.CHAPTER_3.MY_STAGE/marketing_campaign.csv.gz")\
.copy_into_table("PURCHASE_HISTORY")

In [8]:
session.table("PURCHASE_HISTORY").show() 

-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"ID"  |"YEAR_BIRTH"  |"EDUCATION"  |"MARITAL_STATUS"  |"INCOME"  |"KIDHOME"  |"TEENHOME"  |"DT_CUSTOMER"  |"RECENCY"  |"MNTWINES"  |"MNTFRUITS"  |"MNTMEATPRODUCTS"  |"MNTFISHPRODUCTS"  |"MNTSWEETPRODUCTS"  |"MNTGOLDPRODS"  |"NUMDEALSPURCHASES"  |"NUMWEBPURCHASES"  |"NUMCATALOGPURCHASES"  |"NUMSTOREPURCHASES"  |"NUMWEBVISITSMONTH"  |
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------